In [ ]:
# Overall Built by Gemini
import requests
import time
import pandas as pd

class SimFinAPI:
    # A wrapper for the SimFin API to retrieve financial data.

    def __init__(self, api_key):
        #Initializes the SimFinAPI wrapper.
        #Args:
        #    api_key (str): Your SimFin API key.
        self.api_key = api_key
        self.base_url = "https://simfin.com/api/v3/"
        self.headers = {"Authorization": f"api-key {self.api_key}"}

    def get_simfin_id(self, ticker):
      endpoint = f"companies/list"
      try:
          response = requests.get(self.base_url + endpoint, headers=self.headers)
          response.raise_for_status()
          data = response.json()
          for company in data:
            if company['ticker'] == ticker:
              return company['simFinId']
          return None
      except requests.exceptions.RequestException as e:
          print(f"Error fetching SimFinId for {ticker}: {e}")
          return None

    def get_share_prices(self, simfin_id, start_date="2020-01-01", end_date="2023-12-31"):
        endpoint = f"companies/id/{simfin_id}/prices"
        params = {"start": start_date, "end": end_date}
        #rest of function is the same

    def get_shares_outstanding(self, simfin_id):
        endpoint = f"companies/common-shares-outstanding"
        params = {"stype": "bs", "ptype": "MRQ"}
        #rest of function is the same

    def get_data(self, ticker):
        simfin_id = self.get_simfin_id(ticker)
        if simfin_id is None:
          return None
        prices_df = self.get_share_prices(simfin_id)
        shares_outstanding = self.get_shares_outstanding(simfin_id)
        #rest of function is the same

# Example usage:
if __name__ == "__main__":
    api_key = "c63dcc1f-1331-4513-97b0-9cb6ee877517"
    simfin_api = SimFinAPI(api_key)

    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA"]

    for ticker in tickers:
        data = simfin_api.get_data(ticker)
        if data is not None:
            print(f"Data for {ticker}:\n{data.head()}\n")
        time.sleep(0.6)

Error fetching SimFinId for AAPL: 404 Client Error: Not Found for url: https://www.simfin.com/api/v1/companies/list
Error fetching SimFinId for MSFT: 404 Client Error: Not Found for url: https://www.simfin.com/api/v1/companies/list
Error fetching SimFinId for GOOGL: 404 Client Error: Not Found for url: https://www.simfin.com/api/v1/companies/list
Error fetching SimFinId for AMZN: 404 Client Error: Not Found for url: https://www.simfin.com/api/v1/companies/list
Error fetching SimFinId for NVDA: 404 Client Error: Not Found for url: https://www.simfin.com/api/v1/companies/list


In [ ]:
# My Experimental Code after reading the API instructions

import requests

url = "https://backend.simfin.com/api/v3/companies/common-shares-outstanding"

headers = {"accept": "application/json", "Authorization": f"api-key c63dcc1f-1331-4513-97b0-9cb6ee877517"}
params = {"ticker": "AAPL","start":"2020-01-01","end":"2020-10-01"}
response = requests.get(url, headers=headers, params = params)

print(response.text)

# Main issues - the shares outstanding are gathered on irregular dates, and so you can't ask for a specific date, or even within a week, because the number is likely to not exist. 
# One way to avoid this - get the final value, or most recent. 
# Solution - grab the -1 value in the list, and then take the valeu under the 'value' key
# Attempt 2

[{"pid":111052,"endDate":"2020-01-17","value":17501920000},{"pid":111052,"endDate":"2020-03-28","value":17295948000},{"pid":111052,"endDate":"2020-04-17","value":17337340000},{"pid":111052,"endDate":"2020-06-27","value":17135756000},{"pid":111052,"endDate":"2020-07-17","value":17102536000},{"pid":111052,"endDate":"2020-09-26","value":16976763000}]


In [ ]:
# Attempt 2 - PROBLEM SOLVED - we extracted the latest shares outstanding.
import requests

url = "https://backend.simfin.com/api/v3/companies/common-shares-outstanding"

headers = {"accept": "application/json", "Authorization": f"api-key c63dcc1f-1331-4513-97b0-9cb6ee877517"}
params = {"ticker": "AAPL"}
response = requests.get(url, headers=headers, params = params)
response = response.json()
answer = response[-1]['value']
print(answer)

15022073000


In [16]:
# Now onto the prices


url = "https://backend.simfin.com/api/v3/companies/prices/compact"

headers = {"accept": "application/json", "Authorization": f"api-key c63dcc1f-1331-4513-97b0-9cb6ee877517"}
params = {"ticker": "AAPL","start":"2020-01-01","end":"2020-10-01"}
response = requests.get(url, headers=headers, params = params)
response = response.json()

print(response)

[{'columns': ['Date', 'Dividend Paid', 'Common Shares Outstanding', 'Last Closing Price', 'Adjusted Closing Price', 'Highest Price', 'Lowest Price', 'Opening Price', 'Trading Volume'], 'name': 'APPLE INC', 'id': 111052, 'ticker': 'AAPL', 'currency': 'USD', 'isin': 'US0378331005', 'data': [['2020-01-02', None, 17539836000, 75.09, 72.72, 75.15, 73.8, 74.06, 135647456], ['2020-01-03', None, 17539836000, 74.36, 72.01, 75.14, 74.12, 74.29, 146535512], ['2020-01-06', None, 17539836000, 74.95, 72.58, 74.99, 73.19, 73.45, 118578576], ['2020-01-07', None, 17539836000, 74.6, 72.24, 75.22, 74.37, 74.96, 111510620], ['2020-01-08', None, 17539836000, 75.8, 73.41, 76.11, 74.29, 74.29, 132363784], ['2020-01-09', None, 17539836000, 77.41, 74.96, 77.61, 76.55, 76.81, 170486168], ['2020-01-10', None, 17539836000, 77.58, 75.13, 78.17, 77.06, 77.65, 140869088], ['2020-01-13', None, 17539836000, 79.24, 76.74, 79.27, 77.79, 77.91, 120114968], ['2020-01-14', None, 17539836000, 78.17, 75.7, 79.39, 78.04, 79.1